In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%cd /kaggle/working/
!git clone https://github.com/valkryhx/chatGLM-6B-QLoRA

In [1]:
import torch
a=[i for i in range(24)]
ta=torch.tensor(a)
ta=ta.view(2,3,4)
ta

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])

In [2]:
ta[:,-1]

tensor([[ 8,  9, 10, 11],
        [20, 21, 22, 23]])

In [4]:
ta[:,-1].split(1,dim=0)

(tensor([[ 8,  9, 10, 11]]), tensor([[20, 21, 22, 23]]))

In [6]:
x,y=ta[-1].split(1,dim=0)

ValueError: too many values to unpack (expected 2)

In [ ]:
#cat /kaggle/working/chatGLM-6B-QLoRA/output-rm-1k-0812-v1/checkpoint-40/adapter_config.json

In [7]:
%cd chatGLM-6B-QLoRA
!git pull --all --force 
!pip install -r requirements.txt

/kaggle/working/chatGLM-6B-QLoRA
Fetching origin
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 1.67 KiB | 285.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   ae1bef3..5340192  main       -> origin/main
Updating ae1bef3..5340192
Fast-forward
 ppo_chatglm2.py | 6 +++---
 rm_3.py         | 7 ++++++-
 2 files changed, 9 insertions(+), 4 deletions(-)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 12.7 MB/s eta 0:00:0000:0100:01
   

# 

In [ ]:
cat /kaggle/working/chatGLM-6B-QLoRA/output-rm-1k-0811-v3/config.json

In [ ]:
!git pull --all --force
!CUDA_VISIBLE_DEVICES=0 python  rm_trl.py \
--model_name 'THUDM/chatglm2-6b' \
--resume_from_checkpoint /kaggle/working/chatGLM-6B-QLoRA/reward_model_0809_v1/checkpoint-50 \
--num_train_epochs 2 \
--gradient_accumulation_steps 1 \
--per_device_train_batch_size 4 \
--per_device_eval_batch_size  4 \
--max_length 512 \
--output_dir ./reward_model_0810_v2 \
--train_subset 80 \
--eval_subset 20 \
--local_rank 0  \
--bf16 False

# 上面的warning There were missing keys in the checkpoint model loaded: 'transformer.embedding.word_embeddings.weight', 'transformer.rotary_pos_emb.inv_freq', 'transformer.encoder.layers.0.input_layernorm.weight', ....
# 是 trainingArguments中设置 load_best_model_at_end = True 后出现的
# 说明 trainer的确在最后把最好的那个adapters的参数做了load（只不过load时strict=True要求严格匹配了）
# 这说明保存在output_dir中的pytorch_model.bin和vhead.bin参数都是最佳的 我们来验证下

验证成功  跑的是py中的test_load_best()


# deepspeed 单机多卡从头训练 reward model

In [ ]:
!git pull --all --force 
!deepspeed --include localhost:0,1  rewardmodel_qlora_chatglm2.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0811-v2 \
  --num_train_samples 200 \
  --num_eval_samples 100 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 800 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 4  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  2  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json


--deepspeed ds_zero2_config.json

# 单机多卡deepspeed 从ckpt继续训练 

In [ ]:
!git pull --all --force 
!deepspeed --include localhost:0,1  rewardmodel_qlora_chatglm2.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0811-v3 \
  --resume_from_checkpoint output-rm-1k-0811-v2 \
  --num_train_samples 200 \
  --num_eval_samples 100 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 800 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 4  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  2  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json

In [ ]:
ls -lrt /kaggle/working/chatGLM-6B-QLoRA/output-rm-1k-0811-v3

# 单卡训练 reward model

In [ ]:
!git pull --all --force 
!CUDA_VISIBLE_DEVICES=0 python  rewardmodel_qlora_chatglm2.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0811-v1 \
  --num_train_samples 20 \
  --num_eval_samples 20 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 800 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 4  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  10  \
  --save_total_limit 2 \
  --load_in_4bit True 


In [ ]:
!git pull --all --force 
!CUDA_VISIBLE_DEVICES=0 python  rm_3.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0812-v1 \
  --num_train_samples 50 \
  --num_eval_samples 50 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 500 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 1 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  4  \
  --save_total_limit 2 \
  --load_in_4bit True 

In [ ]:
rm -rf output-rm-1k-0812-v1

参考 https://github.com/hiyouga/ChatGLM-Efficient-Tuning/blob/main/src/glmtuner/tuner/core/adapter.py#L83
实现 resume——from --checkpoint

参考

In [ ]:
https://huggingface.co/docs/trl/models#trl.AutoModelForCausalLMWithValueHead
    https://github.com/yongzhuo/chatglm-maths/blob/main/chatglm_maths/t10_lora_trl_train_ppo.py#L175
        https://huggingface.co/docs/trl/v0.5.0/en/customization#use-the-cuda-cache-optimizer
            https://huggingface.co/docs/trl/trainer#trl.RewardTrainer

# python 【rm_3.py】 单卡训练 从头开始

In [ ]:
!git pull --all --force 
!CUDA_VISIBLE_DEVICES=0 python  rm_3.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-rm-1k-0813-v1 \
  --num_train_samples 20 \
  --num_eval_samples 20 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 400 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  2e-5 \
  --num_train_epochs  4  \
  --save_total_limit 2 \
  --load_in_4bit True 

# deepspeed train 【rm_3.py】恢复训练 
#  batach_size 2  gradient accumulation steps 4

In [15]:
rm -rf output-rm-1k-0812-v2

In [ ]:
--resume_from_checkpoint output-rm-1k-0812-v1/checkpoint-60 \

In [18]:
!git pull --all --force 
!deepspeed --include localhost:0,1   rm_3.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --resume_from_checkpoint output-rm-1k-0812-v2 \
  --output_dir output-rm-1k-0812-v3 \
  --num_train_samples 100 \
  --num_eval_samples 50 \
  --train_data_path ./data/rm_data  \
  --eval_data_path  ./data/rm_data    \
  --data_type sharegpt  \
  --max_length 400 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2  \
  --gradient_accumulation_steps 4 \
  --learning_rate  2e-5 \
  --num_train_epochs  8  \
  --save_total_limit 2 \
  --load_in_4bit True 

Fetching origin
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), 1.93 KiB | 152.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   7f84c1a..10b5810  main       -> origin/main
Updating 7f84c1a..10b5810
Fast-forward
 rm_3.py | 10 +++++-----
 1 file changed, 5 insertions(+), 5 deletions(-)
[2023-08-14 12:49:32,249] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to o

In [ ]:
ls output-rm-1k-0812-v2

In [ ]:
ls /kaggle/working/chatGLM-6B-QLoRA/data/rlhf-reward-single-round-trans_chinese/

# PPO training 

In [ ]:
!pip install git+https://github.com/huggingface/trl.git

In [12]:
pwd

'/kaggle/working/chatGLM-6B-QLoRA'

# PPO能运行！千辛万苦改代码成功

In [87]:
!git pull --all --force
!CUDA_VISIBLE_DEVICES=0,1 python ppo_chatglm2.py \
    --base_model_name THUDM/chatglm2-6b \
    --merged_sft_model_path THUDM/chatglm2-6b \
    --sft_model_lora_path output-rm-1k-0812-v3 \
    --reward_model_lora_path output-rm-1k-0812-v3 \
    --adafactor False \
    --save_freq 2 \
    --output_max_length 128 \
    --batch_size 1 \
    --gradient_accumulation_steps 1 \
    --batched_gen True \
    --ppo_epochs 2 \
    --seed 0 \
    --learning_rate 1e-5 \
    --early_stopping True \
    --output_dir ppo_0813_v1 \
    --log_with tensorboard

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 737 bytes | 368.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   c152e02..0d44ebd  main       -> origin/main
Updating c152e02..0d44ebd
Fast-forward
 ppo_chatglm2.py | 1 +
 1 file changed, 1 insertion(+)
[2023-08-14 17:28:47,349] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorfl

In [74]:
all_values=[]
a=torch.tensor([[35.2010],
        [56.9074],
        [ 1.1382],
        [ 0.2368],
        [ 2.2143]]).permute(1,0)
b=torch.tensor([[35.2010],
        [56.9074],
        [ 1.1382],
        [ 0.2368],
        [ 2.2143]]).permute(1,0)
all_values.append(a)
all_values.append(b)

In [75]:
all_values

[tensor([[35.2010, 56.9074,  1.1382,  0.2368,  2.2143]]),
 tensor([[35.2010, 56.9074,  1.1382,  0.2368,  2.2143]])]

In [81]:
 torch.cat(all_values)[:, :-1]

tensor([[35.2010, 56.9074,  1.1382,  0.2368],
        [35.2010, 56.9074,  1.1382,  0.2368]])

In [65]:
scores=[torch.tensor(-1.0057, device='cuda:0')]
scores=[t.cpu() .numpy() for t in scores]                     
#scores = torch.tensor(scores).to(self.current_device)
#scores=torch.tensor(scores)
scores

[array(-1.0057, dtype=float32)]

In [56]:
score=torch.tensor([26.9585, 54.7772,  0.3836,  1.8755,  1.4875,  0.6672, -1.2902, -1.1666,
        -3.4029,  2.9045,  2.2146, -0.4719, -1.1094, -3.8322, -0.4235,  2.3752,
         2.2033,  5.0141,  7.3238,  2.8239,  4.4018,  6.8154,  4.1981,  1.3292,
         6.6918,  5.5280,  3.6042,  3.8058,  3.5440,  1.3948,  1.1925,  1.9285,
         3.5308,  7.6298,  7.2233,  7.8028, 10.3878,  2.4796,  6.6148,  3.1376,
         2.6324,  5.2144,  1.2815,  5.0430,  2.4158,  3.6837,  8.1965,  3.5462,
         0.8770, 13.3891,  3.6978,  7.1916, -0.8622,  2.6911, -0.7917, -1.9702,
         7.8929,  6.5023,  4.2341, -3.7883, -2.2751,  4.5737,  2.5624,  0.2927,
        -1.7391,  5.1636, -1.0061])

In [58]:
torch.tensor([-0.], device='cuda:0')+score[-1]

tensor([-1.0061], device='cuda:0')

In [42]:
import torch
scores=[torch.tensor([26.9585, 54.7772,  0.3836,  1.8755,  1.4875,  0.6672, -1.2902, -1.1666,
        -3.4029,  2.9045,  2.2146, -0.4719, -1.1094, -3.8322, -0.4235,  2.3752,
         2.2033,  5.0141,  7.3238,  2.8239,  4.4018,  6.8154,  4.1981,  1.3292,
         6.6918,  5.5280,  3.6042,  3.8058,  3.5440,  1.3948,  1.1925,  1.9285,
         3.5308,  7.6298,  7.2233,  7.8028, 10.3878,  2.4796,  6.6148,  3.1376,
         2.6324,  5.2144,  1.2815,  5.0430,  2.4158,  3.6837,  8.1965,  3.5462,
         0.8770, 13.3891,  3.6978,  7.1916, -0.8622,  2.6911, -0.7917, -1.9702,
         7.8929,  6.5023,  4.2341, -3.7883, -2.2751,  4.5737,  2.5624,  0.2927,
        -1.7373,  5.1582, -1.0060], device='cuda:0')]

In [48]:
device='cuda:0'
scores=[t.cpu() .numpy() for t in scores]
aa=torch.tensor(s)
aa=aa.to(device)
aa

tensor([[26.9585, 54.7772,  0.3836,  1.8755,  1.4875,  0.6672, -1.2902, -1.1666,
         -3.4029,  2.9045,  2.2146, -0.4719, -1.1094, -3.8322, -0.4235,  2.3752,
          2.2033,  5.0141,  7.3238,  2.8239,  4.4018,  6.8154,  4.1981,  1.3292,
          6.6918,  5.5280,  3.6042,  3.8058,  3.5440,  1.3948,  1.1925,  1.9285,
          3.5308,  7.6298,  7.2233,  7.8028, 10.3878,  2.4796,  6.6148,  3.1376,
          2.6324,  5.2144,  1.2815,  5.0430,  2.4158,  3.6837,  8.1965,  3.5462,
          0.8770, 13.3891,  3.6978,  7.1916, -0.8622,  2.6911, -0.7917, -1.9702,
          7.8929,  6.5023,  4.2341, -3.7883, -2.2751,  4.5737,  2.5624,  0.2927,
         -1.7373,  5.1582, -1.0060]], device='cuda:0')

In [ ]:
import torch
a=[25.256540298461914, 49.58720016479492, 0.18187294900417328, 1.4897446632385254, 1.1968485116958618, 0.10057299584150314, -1.5109386444091797, -1.4273135662078857, -3.3894641399383545, 2.263216972351074, 1.5636731386184692, -1.0434900522232056, -1.8036266565322876, -4.275576591491699, -0.6687140464782715, 2.0810747146606445, 1.6051537990570068, 5.046526908874512, 7.157630920410156, 2.9234297275543213, 4.255917072296143, 6.61237096786499, 4.171794414520264, 1.1586885452270508, 6.537792205810547, 5.545578479766846, 3.443131923675537, 3.6469266414642334, 3.330624580383301, 1.313371181488037, 0.9745500087738037, 1.6175665855407715, 3.1739673614501953, 7.654401779174805, 7.4569196701049805, 8.102145195007324, 10.512571334838867, 2.8823323249816895, 6.398488998413086, 3.0214900970458984, 2.5821034908294678, 5.287378311157227, 1.5696346759796143, 5.16638708114624, 2.148916244506836, 3.395231246948242, 8.202319145202637, 3.30208683013916, 0.7855610847473145, 13.19229507446289, 3.4890899658203125, 6.808653831481934, -0.6579917669296265, 2.4623849391937256, -0.7382142543792725, -2.5757625102996826, 7.459266662597656, 5.911978721618652, 3.8411192893981934, -4.349767684936523, -2.815305233001709, 3.5878658294677734, 1.8798978328704834, -0.4172825813293457, -2.3522253036499023, 4.773746490478516, -0.9820916652679443]
b=torch.tensor(a)
b

In [ ]:
import torch
scores=torch.tensor([[25.2565, 49.5872],[5,6]] ,device='cuda:0')

In [ ]:
a=scores-9

In [ ]:
a

In [ ]:
from dataclasses import dataclass, field
from typing import Optional

import torch
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoTokenizer, HfArgumentParser

In [ ]:
from transformers import AutoModelForSequenceClassification
mymodel = AutoModelForSequenceClassification.from_pretrained("THUDM/chatglm2-6b",trust_remote_code=True)

In [ ]:
!git clone https://github.com/valkryhx/LLaMA-Efficient-Tuning

In [ ]:
cd /kaggle/working/LLaMA-Efficient-Tuning

In [ ]:
!pip install -r requirements.txt

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python src/train_bash.py \
    --stage ppo \
    --model_name_or_path THUDM/chatglm2-6b \
    --do_train \
    --dataset alpaca_gpt4_en \
    --template default \
    --finetuning_type lora \
    --resume_lora_training False \
    --checkpoint_dir /kaggle/working/chatGLM-6B-QLoRA/output-rm-1k-0813-v1/checkpoint-40 \
    --reward_model /kaggle/working/chatGLM-6B-QLoRA/output-rm-1k-0813-v1/checkpoint-40 \
    --output_dir 0808 \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 1000 \
    --learning_rate 1e-5 \
    --num_train_epochs 1.0 \
    --plot_loss